In [1]:
import numpy as np
import cvxpy as cvx
import pandas as pd

In [2]:
# Data for the assets
# US_3-MONTH_T-BILLS US_GOVN_LONG_BONDS SP_500 WILSHIRE_5000
X = np.array([[1984, 1.103, 1.159, 1.061, 1.030],
[1985, 1.080, 1.366, 1.316, 1.326],
[1986, 1.063, 1.309, 1.186, 1.161],
[1987, 1.061, 0.925, 1.052, 1.023],
[1988, 1.071, 1.086, 1.165, 1.179],
[1989, 1.087, 1.212, 1.316, 1.292],
[1990, 1.080, 1.054, 0.968, 0.938],
[1991, 1.057, 1.193, 1.304, 1.342],
[1992, 1.036, 1.079, 1.076, 1.090],
[1993, 1.031, 1.217, 1.100, 1.113],
[1994, 1.045, 0.889, 1.012, 0.999]])
Xdf = pd.DataFrame(X[:,1:5],index=X[:,0].astype(int))
Xdf.columns = ['Tbills', 'Tbonds', 'SP500', 'Wilshire5K']

In [3]:
# mean and covariance from the data frame
mudf = Xdf.mean()
mu = np.array(mudf)

Vdf = Xdf.cov()
V = np.array(Vdf)

# target return 
R = mu.sum()/len(mu)

# short limit
K = 1

In [4]:
np.set_printoptions(precision=5)
print(V)

[[0.00049 0.00083 0.00058 0.00032]
 [0.00083 0.02169 0.01328 0.01417]
 [0.00058 0.01328 0.01572 0.01707]
 [0.00032 0.01417 0.01707 0.019  ]]


We want to solve the optimization problem
$$
\begin{array}{rl}
\mbox{min} & x^\top V x\\
\mbox{s.t.} & \mu^\top x = R,\\
& \mathbf{1}^\top x = 1,\\
& x \geq -K \mathbf{1}
\end{array}
$$

In [7]:
# equality constraints
d = len(mu)
A = np.matrix([mu, np.ones(d)])
b = np.matrix([[R],[1]])
meq = len(b)

# inequality constraints
H = np.matrix(np.identity(d))
g = -K*np.ones(d)

# create an initial feasible solution
# diversify between the assets with the highest and the lowest returns
mumax = mu.max()
kmax = mu.argmax()

mumin = mu.min()
kmin = mu.argmin()
x = np.zeros(len(mu))

x[kmax] = (R-mumin)/(mumax- mumin)
x[kmin] = (mumax-R)/(mumax-mumin)

# gradient at the x
gradfx = (2*V).dot(x)

In [149]:
# find the active constraints
tol = 1e-6

I = list(x < -K + tol)
if (True in I):
    Hx = H[I,:]
    Aeq = np.concatenate((A, Hx),axis=0)   # concatenate original set of equality constraints with active constraints
else:
    Aeq = A                                # if no constraints are active, only have the initial set of equality
m = np.shape(Aeq)[0]

Solve the system of equations
$$
\underbrace{\begin{bmatrix}
Aeq & \mathbf{0}\\
2V & -Aeq^\top 
\end{bmatrix}}_{M}
\underbrace{\begin{bmatrix}
y \\ v
\end{bmatrix}}_{z}
= \underbrace{\begin{bmatrix}
0 \\
- \nabla f(x)
\end{bmatrix}}_{q}
$$

In [151]:
gradfx = (2*V).dot(x)
M1 = np.concatenate((Aeq,np.zeros((m,m))),axis=1)
M2 = np.concatenate((2*V,-Aeq.T),axis=1)
M = np.concatenate((M1,M2),axis=0)

q = np.concatenate((np.zeros(m),-gradfx),axis=0)
z = np.linalg.solve(M,q)
y = z[0:d]
v = z[d:d+m]

if (np.linalg.norm(y) > tol):
    # take a step
    step = 1;
    for i in range(np.shape(H)[0]): # all inequality constraints
        if (H[i,:].dot(y) < -tol):
            step = min(step, (g[i]-H[i,:].dot(x))/H[i,:].dot(y))
    x = x + float(step)*y;
    drop = False
else: # norm(y) = 0
    # look at the dual variables corresponding to the inquality constraints
    vmin = min(v[meq:m])
    kmin = v[meq:m].argmin()
    if (vmin < -tol): # if the minimum is negative remove the constraint
        print('Dropping constraints')
        drop = True
        Aeq = np.delete(Aeq,meq + kmin, 0)
        m = np.shape(Aeq)[0]
    else:
        print('Optimal solution found')
        optfound = True
print('search direction = ')
np.set_printoptions(precision=3)
print(y)



search direction = 
[  1.395e-01  -1.431e-03   1.725e+00  -1.863e+00]


In [152]:
M1

matrix([[ 1.065,  1.135,  1.141,  1.136,  0.   ,  0.   ],
        [ 1.   ,  1.   ,  1.   ,  1.   ,  0.   ,  0.   ]])

In [153]:
# put it all in a loop

# Initial solution
# create an initial feasible solution
mumax = mu.max()
kmax = mu.argmax()

mumin = mu.min()
kmin = mu.argmin()
x = np.zeros(len(mu))

x[kmax] = (R-mumin)/(mumax- mumin)
x[kmin] = (mumax-R)/(mumax-mumin)

# define an indicator variable for optimal
optfound = False

# indicator variable for drop state
drop = False

while (not optfound):
    
    if (not drop):
        gradfx = (2*V).dot(x)
        I = list(x < -K + tol)
        if (True in I):
            Hx = H[I,:]
            Aeq = np.concatenate((A, Hx),axis=0)
        else:
            Aeq = A
        m = np.shape(Aeq)[0]

    M1 = np.concatenate((Aeq,np.zeros((m,m))),axis=1)
    M2 = np.concatenate((2*V,-Aeq.T),axis=1)
    M = np.concatenate((M1,M2),axis=0)

    q = np.concatenate((np.zeros(m),-gradfx),axis=0)
    z = np.linalg.solve(M,q)
    y = z[0:d]
    v = z[d:d+m]

    if (np.linalg.norm(y) > tol):
        # take a step
        drop = False
        step = 1;
        for i in range(np.shape(H)[0]): # all inequality constraints
            if (H[i,:].dot(y) < -tol):
                step = float(min(step, (g[i]-H[i,:].dot(x))/H[i,:].dot(y)))
        x = x + step*y;
        
        np.set_printoptions(precision=3)
        print('search direction: ' + str(y))
        print('step: ' + str(step))
        print('current iterate: ' + str(x))
        
    else: # norm(y) = 0
        # look at the dual variables corresponding to the inquality constraints
        if (m > meq):
            vmin = min(v[meq:m])
            kmin = v[meq:m].argmin()
            if (vmin < -tol): # if the minimum is negative remove the constraint
                drop = True
                print('Dropping constraints')
                Aeq = np.delete(Aeq,meq + kmin, 0)
                m = np.shape(Aeq)[0]
            else:
                print('Optimal solution found')
                print('Optimal portfolio: ' + str(x))
                optfound = True
        else:
            print('Optimal solution found')
            print('Optimal portfolio: ' + str(x))
            optfound = True
    

search direction: [  1.395e-01  -1.431e-03   1.725e+00  -1.863e+00]
step: 0.5367343398176345
current iterate: [  3.635e-01  -7.681e-04   1.637e+00  -1.000e+00]
search direction: [-0.003  0.035 -0.033  0.   ]
step: 1.0
current iterate: [ 0.361  0.035  1.605 -1.   ]
Optimal solution found
Optimal portfolio: [ 0.361  0.035  1.605 -1.   ]


In [154]:
# cvx solution

z = cvx.Variable(d)

objective = cvx.Minimize(cvx.quad_form(z,V))
constraints = [mu.T*z == R, sum(z) == 1, z >= -K]

prob = cvx.Problem(objective,constraints)
prob.solve()

print('Optimal portfolio')
print(z.value.T)


Optimal portfolio
[[ 0.361  0.035  1.605 -1.   ]]
